# Was sind die größten und komplexesten Methoden?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [5]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [6]:
import pandas as pd

query ="MATCH (t:Type)-[:HAS_SOURCE]->(f), (t)-[:DECLARES]->(m:Method) RETURN t.fqn as type, m.signature as signature,  sum(m.effectiveLineCount) as loc, sum(m.cyclomaticComplexity) as complexity"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [7]:
df.head()

,complexity,loc,signature,type
0,1,2,void <init>(),org.junit.rules.TestWatchmanTest$FailingTest
1,2,2,java.lang.Class getType(),org.junit.experimental.theories.ParameterSigna...
2,1,2,void forgetMax(),org.junit.tests.experimental.max.JUnit38Sortin...
3,0,0,void run(junit.framework.TestResult),junit.tests.extensions.ExtensionTest$TornDown
4,1,6,void doesNotConsiderParametersWhichChangedAfte...,org.junit.runners.parameterized.TestWithParame...


Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Anzahl der Quelltextzeilen (loc) sortiert und die ersten zehn Methoden extrahiert. Die Werte werden in den Variablen <font face="Courier">x_loc</font> und <font face="Courier">y_method_loc</font> gespeichert.

In [8]:
dfm=df.sort_values('loc',ascending=False)
dfm=dfm.head(10)
def x_loc(liste_x):
    liste_x=dfm['loc'].tolist()
    return liste_x
def y_method_loc(y):
    liste_y=[]
    y=dfm['signature'].tolist()
    for i in y:
        signature=i.split(' ')[-1]
        liste_y.append(signature)
    return liste_y

Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Komplexität (complexity) sortiert und die ersten zehn Methoden extrahiert. Die Werte werden in den Variablen <font face="Courier">x_complexity</font> und <font face="Courier">y_method_complexity</font> gespeichert.

In [9]:
dfc=df.sort_values('complexity',ascending=False)
dfc=dfc.head(10)

def x_complexity(x):
    liste_x=dfc['complexity'].tolist()
    return(liste_x)

def y_method_complexity(y):
    liste=[]
    y=dfc['signature'].tolist()
    for i in y:
        g=i.split(' ')[-1]
        liste.append(g)
    return liste

## 4. Visualisierung
Die Daten werden mittels eines Bar Charts visualisiert (https://plot.ly/python/horizontal-bar-charts/, x=x_loc, y=y_method_loc).

In [10]:
from IPython.display import display, HTML

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

Die Daten werden mittels eines Bar Charts visualisiert (https://plot.ly/python/horizontal-bar-charts/, x=x_complexity, y=y_method_complexity).

# Größte Methoden
Im Balkendiagramm Größte Methoden werden die zehn Methoden mit den meisten Quelltextzeilen in Abhängigkeit der absoluten Anzahl ihrer Quelltextzeilen dargestellt. Die größte Methode wird somit ermittelt.** DIe Methode, die am fehleranfälligsten ist. ???**


In [11]:
import pygal 
line_chart = pygal.HorizontalBar()
line_chart.title = 'Komplexeste Methoden'
line_chart.x_title='zyklomatische Komplexität'
line_chart.y_title='Methode'
line_chart.show_legend=False
line_chart.x_labels= y_method_complexity(dfc)
line_chart.add('Programm 1',x_complexity(dfc))
display(HTML(base_html.format(rendered_chart=line_chart.render(is_unicode=True))))


In [12]:
import pygal 
line_chart = pygal.HorizontalBar()
line_chart.title = 'Größte Methoden'
line_chart.x_title='Anzahl der Quelltextzeilen'
line_chart.y_title='Methode'
line_chart.show_legend=False
line_chart.x_labels= y_method_loc(dfm)
line_chart.add('Programm 1',x_loc(dfm))
display(HTML(base_html.format(rendered_chart=line_chart.render(is_unicode=True))))

In [13]:
a=df.sort_values('loc',ascending=False)
b=a.head(10)
def x_loc(x):
    x=b['loc'].tolist()
    return x
def y_method_loc(y):
    liste=[]
    y=b['signature'].tolist()
    for i in y:
        g=i.split(' ')[-1]
        liste.append(g)
    return liste

#Visualisierung
import pygal 
line_chart = pygal.HorizontalBar()
line_chart.title = 'Größte Methoden'
line_chart.x_title='Anzahl der Quelltextzeilen'
line_chart.y_title='Methode'
line_chart.show_legend=False
line_chart.x_labels= y_method_loc(b)
line_chart.add('Programm 1',x_loc(b))
display(HTML(base_html.format(rendered_chart=line_chart.render(is_unicode=True))))

## Größte Methoden

In [17]:
# Sortiere Zeilen des Dataframes aufsteigend ihrer Anzahl der Commits in neuem Dataframe.
sorted_loc=df.sort_values('loc',ascending=False)
#  Speichert die ersten 10 Zeilen dieses Dataframes in neuem Dataframe.
top10_loc=sorted_loc.head(10)

# Hole Werte aus Spalten aus Dataframe. Umwandlung dieser Werte in list Objekt.
x_loc=top10_loc['loc'].tolist()
y_method_loc=top10_loc['signature'].tolist()

# Sortiert die Werte der Listen absteigend der Anzahl der Commits. 
x_loc.reverse()
y_method_loc.reverse()


In [19]:
import pygal 
bar_chart = pygal.HorizontalBar()
bar_chart.title = 'Größte Methoden'
bar_chart.x_title='Anzahl der Quelltextzeilen'
bar_chart.y_title='Methode'
bar_chart.show_legend=False
bar_chart.x_labels= y_method_loc
bar_chart.add('Programm 1',x_loc)
display(HTML(base_html.format(rendered_chart=bar_chart.render(is_unicode=True))))

## Komplexeste Methoden

In [20]:
# Sortiere Zeilen des Dataframes aufsteigend ihrer Anzahl der Commits in neuem Dataframe.
sorted_complexity=df.sort_values('loc',ascending=False)
#  Speichert die ersten 10 Zeilen dieses Dataframes in neuem Dataframe.
top10_complexity=sorted_complexity.head(10)

# Hole Werte aus Spalten aus Dataframe. Umwandlung dieser Werte in list Objekt.
x_complexity=top10_complexity['loc'].tolist()
y_method_complexity=top10_complexity['signature'].tolist()

# Sortiert die Werte der Listen absteigend der Anzahl der Commits. 
x_complexity.reverse()
y_method_complexity.reverse()


In [21]:
import pygal 
bar_chart = pygal.HorizontalBar()
bar_chart.title = 'Komplexeste Methoden'
bar_chart.x_title='zyklomatische Komplexität'
bar_chart.y_title='Methode'
bar_chart.show_legend=False
bar_chart.x_labels= y_method_complexity
bar_chart.add('Programm 1',x_complexity)
display(HTML(base_html.format(rendered_chart=bar_chart.render(is_unicode=True))))